# Umělé neuronové sítě typu MLP


## Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def show_loss(iterations, loss, epoch=False):
    plt.plot(iterations, loss)
    plt.xlabel("Epocha" if epoch else "Iterace")
    plt.ylabel("Loss")
    plt.title("Průběh trénování")
    plt.show()

npzfile = np.load('data/data_10.npz')
npzfile.files


In [ ]:
x = npzfile['x']
xTest = npzfile['xTest']

y = npzfile['y']
yTest = npzfile['yTest']

print(f"{x.shape=}")
print(f"{y.shape=}")
print(f"Počet tříd: {np.max(y) + 1}")

w1test = npzfile['w1']
w2test = npzfile['w2']

## Stavební bloky sítě

### Funkce sigmoid

$$ sigmoid(u) = \sigma (u) = \frac{e^u}{1+e^u} = \frac{1}{1+e^{-u}} $$


In [ ]:
def sigmoid(u):
    #################################################################
    # ZDE DOPLNIT
    ...
    #################################################################


In [ ]:
#Kontrola:
u = np.array([[1,2],[-3,-4]])
sigmoid(u)

#### Derivace funkce sigmoid:
$$ \sigma' (u) = \sigma (u) (1 - \sigma(u)) $$

In [ ]:
def sigmoid_grad(u):
    
    #################################################################
    # ZDE DOPLNIT
    ...
    #################################################################


In [ ]:
#Kontrola:
sigmoid_grad(u)

### ReLU

$$ f(u) = max(0, u) $$



In [ ]:
def relu(u):
    #################################################################
    # ZDE DOPLNIT
    ...
    #################################################################


In [ ]:
#Kontrola:
relu(u)

#### Derivace funkce ReLU:
$$ f'(x) = \boldsymbol{1} (x \ge 0)$$

Derivace přímo v bodě nula je dodefinována na hodnotu nula.

Gradient se přes tento blok přenáší:
1) Nezměněný, pokud je hodnota na vstupu z dopředného průchodu větší než nula.
2) Přenesená hodnota je nula, pokud je hodnota na vstupu z dopředného průchodu menší nebo rovna nule.

In [ ]:
def relu_grad(u):
    #################################################################
    # ZDE DOPLNIT
    ...
    #################################################################


In [ ]:
#Kontrola:
relu_grad(u)

### One Hot Encoding
$ \pi $ nabývá hodnoty 1 pouze pro jednu třídu. Např. máme celkem 3 třídy (0, 1, 2): $\pi_0 = [0,1,0]$  pro $y_0 = 1$


$$
    classes = 
        \begin{bmatrix}
        1 \\
        0 \\
        2\\
        1 \\
        \end{bmatrix} 
    \implies
        \pi = 
        \begin{bmatrix}
        0 & 1 & 0 \\
        1 & 0 & 0 \\
        0 & 0 & 1 \\
        0 & 1 & 0 \\
        \end{bmatrix} 
$$

In [ ]:
def one_hot_encoding(data):
    #################################################################
    # ZDE DOPLNIT
    ...
    #################################################################


In [ ]:
#Kontrola:
encoded = one_hot_encoding(y)
encoded[[0,900,1800,2700,3500,4200],:]

### Softmax

- Funkce softmax má c vstupů a c výstupů. 
- Všechny výstupy jsou kladná čísla. 
- Součet všech výstupů dohromady je roven číslu 1.
$$\widehat{y_c} = softmax(u) = \frac{e^{u_c}}{\sum_{d=0}^{c} {e^{u_d}}} $$


In [ ]:
def softmax(u):
    """
    softmax !radkove!
    """
    #################################################################
    # ZDE DOPLNIT
    ...
    #################################################################


In [ ]:
#Kontrola:
softmax(u)

In [ ]:
def theta_grad(grad_on_output, input_data):
    #################################################################
    # ZDE DOPLNIT
    ...
    #################################################################
    return weight_grad, bias_grad

In [ ]:
#test vypoctu gradientu pro matici vah a biasy

#dva vstupni vektory, kazdy ma 4 hodnoty, cili jde o vrstvu, kde kazdy neuron ma 4 vstupy
input_test = np.array([[7,8,4,1],[9,10,4,2]])
print("input_test")
print(input_test.shape)

#dva gradienty na vystupu, kazdy ma 3 hodnoty, cili jde o vrstvu, ktera ma 3 neurony [a kazdy ma 4 vstupy])
grad_on_output_test = np.array([[1,2,3],[4,2,6]])
print("grad_on_output_test")
print(grad_on_output_test.shape)

w_grad_test,u_grad_test = theta_grad(grad_on_output_test,input_test)

#gradienu vektoru vah ma tedy rozmery 3*4
print("w_grad_test")
print(w_grad_test.shape)
print(w_grad_test)

#gradient biasu ma 3 hodnoty
print("u_grad")
print(u_grad_test.shape)
print(u_grad_test)

## Sítě typu vícevrstvý perceptron = Multi-Layer Perceptron (MLP)



### Předzpracování dat
Pro trénování neuronových sítí je vhodné provádět standardizaci dat na nulovou střední hodnotu a jednotkový rozptyl.

### Inicializace parametrů (váhových koeficientů)
- Váhy neuronů nesmí být nastaveny na stejné hodnoty (např. 0), aby neměly stejnou hodnotu výstupu a stejný gradient
=>
- Je třeba porušit symetrii:
    - Váhy se inicializují jako malá náhodná čísla (polovina kladná, polovina záporná)
    - V praxi se pro ReLU používá hodnota $randn(n) * sqrt(2.0/n)$, kde n je počet vstupů neuronu
    - Započítání počtu vstupů pak zajišťuje, že neurony s různým počtem vstupů mají výstup se stejným rozptylem hodnot
    - Biasy se inicializují na hodnotu 0 nebo 0.01 (symetrie je již porušena inicializací váhových koeficientů)

### Dopředný průchod
Kroky:
1. $x_1$ je $x$ rozšířená o sloupec bias
2. $u_1 = \theta_1^T x_1$ (vstupní vrstva)
3. $a_1 = ReLU(u_1)$ (aktivační funkce, v kódu použijte obecně `activation_function`)
4. $x_2$ je $a_1$ rozšířená o sloupec bias
5. $u_2 = \theta_2^T x_2$ (skrytá vrstva)
6. $\tilde{y} = softmax(u_2)$ (výstupní vrstva)

Na výstupu vznikne podle zvoleného kritéria chyba či odchylka.

### Zpětný průchod

(Hodnoty z dopředného průhodu $x$, $a_1$ a $u_2$ je vhodné si z dopředného průchodu uložit.)

1. $du_2 = softmax(u_2)-\pi(y)$

2. $dW_2 = du_2 a_1^T$  a  $db_2 = du_2 $

3. $da_1 = W_2^T du_2$

4. $du_1 = da_1 \odot relu'(du_1)$ (v kódu použijte obecně `activation_function_derivation`)

5. $dW_1 = du_1^T x$  a  $db = du_1 $

In [ ]:
class TwoLayerPerceptron:
    def __init__ (self, *, input_layer_size, hidden_layer_size, output_size, activation_function, activation_function_derivation):
        #################################################################
        # ZDE DOPLNIT
        self.w1 = ...
        self.w2 = ...

        ...
        #################################################################
        
    def forward(self, x) -> tuple[np.ndarray, dict]:
        """
        Spočítá predikci na základě aktuálních vah.
        """

        #################################################################
        # ZDE DOPLNIT

        #1. vrstva
        x1 = ... #nezapomente na bias(prvni sloupec) #4500x401
        u1 = ...

        #aktivacni funkce pomocí funkce self.activation_function
        a1 = ... #4500x25

        #2. vrstva (skryta vrstva)
        x2 = ... #4500x26
        u2 = ...

        #vystup po softmaxu
        scores = ... #4500x10 scores pro kazdou tridu            
        
        #cache
        forward_cache = ...
        #################################################################
        
        # forward_cache je dictionary obsahující uložené hodnoty potřebné pro zpětný průchod
        return scores, forward_cache
    
    def backward(self, classes, forward_cache: dict) -> dict:
        """
        Vypočítá gradienty na základě hodnot uložených při dopředném průchodu.
        """

        #################################################################
        # ZDE DOPLNIT

        #pomocí self.scores, self.classes a one_hot_encoding 
        du2 = ...
        
        #pomocí funkce theta_grad
        dw2, db2 = ...
        
        da1 = ...
        
        #pomocí funkce self.activation_function_derivation
        #POZOR: tato funce se musí aplikovat na vstupní hodnotu z dopředného průchodu !!
        du1 = ...
        
        #pomocí funkce theta_grad
        dw1, db1 = ...
        
        gradients = ...
        #################################################################

        # gradients je dictionary obsahující vypočtené gradienty
        return gradients


    def update_weights_gd(self, gradients: dict, alpha, lmbd=0):
        """
        Aktualizuje váhy sítě na základě vypočtených gradientů.
        """

        #################################################################
        # ZDE DOPLNIT
    
        #POZOR: dw2 není gradient celé matice w2 ale pouze její části
        #gradient celé matice w2 pro update vah vznikne vhodným spojením dw2 a db2
        ...
        
        #################################################################
    

    def accuracy(self, x, classes):
        #################################################################
        # ZDE DOPLNIT
        ...
        #################################################################
            


In [ ]:
input_layer_size = 400
hidden_layer_size = 25
output_size = len(np.unique(y))


In [ ]:
#Instance pro odladění:
testTlp = TwoLayerPerceptron(input_layer_size=input_layer_size, hidden_layer_size=hidden_layer_size, output_size=output_size, activation_function=sigmoid, activation_function_derivation=sigmoid_grad)
testTlp.w1 = w1test
testTlp.w2 = w2test
# alpha = 0.0005, lmbd=0

...

Data pro odladění s předchozí instancí po jednom kroku trénování:

In [ ]:
def train_gd(model, x, classes, nIter, alpha=0.00015, lmbd=0):
    """
    Natrénuje síť a vykreslí graf vývoje lossu.
    """

    # Na konci každé iterace vypočtěte cross-entropy loss a ulože ho na odpovídající index v poli.
    loss = np.zeros(nIter)

    #################################################################
    # ZDE DOPLNIT  
    ...
    #################################################################

    show_loss(np.arange(nIter), loss)

Trénování modelu s aktivační funkcí sigmoid:

In [ ]:
tlp = TwoLayerPerceptron(input_layer_size=input_layer_size, hidden_layer_size=hidden_layer_size, output_size=output_size, activation_function=sigmoid, activation_function_derivation=sigmoid_grad)
#################################################################
# ZDE DOPLNIT
...
#################################################################
print(f"sigmoid testovaci mnozina : {accuracy}") 

Trénování modelu s aktivační funkcí ReLU:

In [ ]:
tlp = TwoLayerPerceptron(input_layer_size=input_layer_size, hidden_layer_size=hidden_layer_size, output_size=output_size, activation_function=relu, activation_function_derivation=relu_grad)
#################################################################
# ZDE DOPLNIT
...
#################################################################
print(f"relu testovaci mnozina : {accuracy}")

# Bonus: PyTorch a MNIST

Data: Číslovky z datasetu MNIST z bonusové části 7. cvičení (tam si je můžete prohlédnout).

**Dosáhněte accuracy > 97 %**

Tip: Menší batche napomáhají generalizaci.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
npzfile = np.load('data/data_07_mnist_train.npz') 

data = npzfile['data']
ref = npzfile['ref']

# Převod na objekty knihovny PyTorch
data = torch.Tensor(data)
ref = torch.Tensor(ref).long()

In [ ]:
npzfile = np.load('data/data_07_mnist_test.npz') 

test_data = npzfile['data']
test_ref = npzfile['ref']

# Převod na objekty knihovny PyTorch
test_data = torch.Tensor(test_data)
test_ref = torch.Tensor(test_ref).long()

In [ ]:
# Normalizace hodnot pixelů (na základě statistických odhadů z trénovacích dat)
mean = data.mean()
std = data.std()
data = (data - mean) / std
test_data = (test_data - mean) / std

In [ ]:
# Definice modelu

#################################################################
# ZDE DOPLNIT
input_layer_size = ... 
hidden_layer_size = ...
output_size = ...
#################################################################

model = nn.Sequential(
    nn.Linear(input_layer_size, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, output_size),
    # SOFTMAX se nepřidává! Kriteriální funkce ho počítá sama. Výstupem sítě jsou tzv. logity.
)

print(model)

# Konfigurace hyperparametrů trénování.
# Optimizér provádí zvolenou metodu optimalizace sítě (SGD / SGD + momentum / ADAM / jiné).
# Parametr weight_decay je lmbd z našeho kódu.
#################################################################
# ZDE DOPLNIT
num_epochs = ...
batch_size = ...
optimizer = optim.SGD(model.parameters(), lr=..., weight_decay=..., momentum=...)
# optimizer = optim.Adam(...)
# optimizer = optim.AdamW(...)
#################################################################

# Kriteriální funkce: počítá softmax a cross-entropy loss.
criterion = nn.CrossEntropyLoss()

# Nastaví model do trénovacího módu (důležité pro některé typy vrstev, např. Dropout nebo BatchNorm).
model.train()

losses = np.zeros(num_epochs)
num_batches = data.size(0) // batch_size

for epoch in range(num_epochs):
    # Zamíchání dat
    permutation = torch.randperm(data.size(0))
    
    epoch_loss = 0.0
    
    # Smyčka přes minibatche
    for i in range(0, data.size(0), batch_size):
        indices = permutation[i : i + batch_size]
        batch_x, batch_y = data[indices], ref[indices]
        
        # Gradienty jsou uloženy u vah, kterých se týkají. Je potřeba je v každé iteraci explicitně vynulovat, jinak se akumulují.
        optimizer.zero_grad()
        
        # Ekvivalent naší funkce forward (bez softmax).
        logits = model(batch_x)
        
        # Výpočet průměrného lossu podle zvolené kriteriální funkce.
        loss = criterion(logits, batch_y)
        
        # Spočítá gradienty. Ty nejsou výstupem funkce, ale jsou uloženy u vah, kterých se týkají.
        loss.backward()
        
        # Aktualizuje váhy podle zvolené metody.
        optimizer.step()
        
        epoch_loss += loss.item()

    # Průměrný loss.
    losses[epoch] = epoch_loss / num_batches
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {losses[epoch]:.4f}")

show_loss(np.arange(num_epochs), losses, epoch=True)

In [ ]:
# Nastaví model do testovacího módu (důležité pro některé typy vrstev, např. Dropout nebo BatchNorm).
model.eval()

# torch.no_grad() vypne operace pro výpočet gradientu, které se provádí na pozadí, protože je zde nepotřebujeme.
with torch.no_grad():
    logits = model(test_data)
    predicted_classes = logits.argmax(1)
    acc = (predicted_classes == test_ref).float().mean()

print(f"Accuracy: {acc * 100}%")